In [1]:
import os, sys

fileDir = os.getcwd()
path = os.path.join(fileDir, "..")
sys.path.append(path)

from utilities import *

In [2]:
session = create_spark_session("Neo4j GitHub", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


In [3]:
common_URI = "hdfs://namenode:9000//data-team/"

In [4]:
files = session.read.json(f"{common_URI}files.json")

In [5]:
files.printSchema()

root
 |-- id: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- path: string (nullable = true)
 |-- ref: string (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- symlink_target: string (nullable = true)



In [6]:
languages = session.read.json(f"{common_URI}languages.json")

In [7]:
languages.printSchema()

root
 |-- language: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bytes: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [8]:
licences = session.read.json(f"{common_URI}licenses.json")

In [9]:
licences.printSchema()

root
 |-- license: string (nullable = true)
 |-- repo_name: string (nullable = true)



In [10]:
commits = session.read.json(f"{common_URI}commits.json")

In [11]:
commits.show(10)

+--------------------+--------------------+--------------------+----------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              author|              commit|           committer|difference|difference_truncated|encoding|             message|              parent|           repo_name|             subject|             trailer|                tree|
+--------------------+--------------------+--------------------+----------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{{1212055973}, 24...|a564f169dcbd8bd65...|{{1212055973}, 24...|        []|                null|    null|Check in new vers...|[af9132054ca3c08a...|[moses-smt/mosesd...|Check in new vers...|[{da2bf3b6971b326...|4b825dc642cb6eb9a...|
|{{1335874805}, 6f...|01c04e4f313c9d2b0...|{{1335874805}, 6f...|        

In [12]:
commits.printSchema()

root
 |-- author: struct (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- time_sec: string (nullable = true)
 |    |-- tz_offset: string (nullable = true)
 |-- commit: string (nullable = true)
 |-- committer: struct (nullable = true)
 |    |-- date: struct (nullable = true)
 |    |    |-- seconds: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- time_sec: string (nullable = true)
 |    |-- tz_offset: string (nullable = true)
 |-- difference: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- new_mode: string (nullable = true)
 |    |    |-- new_path: string (nullable = true)
 |    |    |-- new_repo: string (nullable = true)
 |    |    |-- new_sha1: string (nullable = true)
 |    |    |-- old_mode: string (nullable = true)
 |    |    

In [13]:
duplicates = commits.where(commits["committer.date.seconds"] != commits["author.date.seconds"])

In [17]:
git_commits= commits.select("commit", "subject", "message", "committer.date", "author.date")

In [18]:
newColumns = ["id","title","message","committer_date", "author_date"]
git_commits=git_commits.toDF(*newColumns)
git_commits.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- message: string (nullable = true)
 |-- committer_date: struct (nullable = true)
 |    |-- seconds: string (nullable = true)
 |-- author_date: struct (nullable = true)
 |    |-- seconds: string (nullable = true)

